In [1]:
import os
import numpy as np

from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt
import pickle


# CONLL

In [2]:
from flair.data import Corpus
from flair.datasets import CONLL_03
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings
from typing import List

corpus: Corpus = CONLL_03(base_path='resources/tasks')


2020-10-02 15:17:33,807 Reading data from resources/tasks/conll_03
2020-10-02 15:17:33,808 Train: resources/tasks/conll_03/eng.train
2020-10-02 15:17:33,809 Dev: resources/tasks/conll_03/eng.testa
2020-10-02 15:17:33,809 Test: resources/tasks/conll_03/eng.testb


In [3]:
flair_corpus=corpus

In [4]:
len(corpus.train)

14987

In [5]:
corpus = [sentence.to_original_text() for sentence in flair_corpus.train]

In [6]:
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

corpus_embeddings = embedder.encode(corpus)

corpus_embeddings_ = np.array(corpus_embeddings)

In [7]:
corpus_embeddings_.shape

(14987, 768)

In [8]:
num_examples = 749

corpus_embeddings = corpus_embeddings_[0:num_examples]

In [9]:
d = 768
nb = corpus_embeddings.shape[0]
nq = corpus_embeddings.shape[0]
xb = corpus_embeddings
xq = corpus_embeddings

In [10]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
749


In [11]:
k = 20                          # we want to see 20 nearest neighbors
D, I = index.search(xq, k)     # actual search

In [12]:
data_dir = 'resources/tasks/conll_03/'
file_path = os.path.join(data_dir, 'sent_id_knn{}.pkl'.format(len(corpus_embeddings)))
pickle.dump( I[:,0:100], open( file_path, "wb" ) )   
file_path        

'resources/tasks/conll_03/sent_id_knn_749.pkl'